In [1]:
%matplotlib inline


Neural Networks
===============

Neural networks can be constructed using the ``torch.nn`` package.

Now that you had a glimpse of ``autograd``, ``nn`` depends on
``autograd`` to define models and differentiate them.
An ``nn.Module`` contains layers, and a method ``forward(input)``\ that
returns the ``output``.

For example, look at this network that classifies digit images:

.. figure:: /_static/img/mnist.png
   :alt: convnet

   convnet

It is a simple feed-forward network. It takes the input, feeds it
through several layers one after the other, and then finally gives the
output.

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or
  weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule:
  ``weight = weight - learning_rate * gradient``

Define the network
------------------

Let’s define this network:



In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


You just have to define the ``forward`` function, and the ``backward``
function (where gradients are computed) is automatically defined for you
using ``autograd``.
You can use any of the Tensor operations in the ``forward`` function.

The learnable parameters of a model are returned by ``net.parameters()``



In [57]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6x6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2,2) window
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, (2,2))
        # If the size is a square you can only specify a single number
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, self.num_flat_features(x)) #QQ: unravel the tensor so it can be fed to a dense net
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) #QQ: need to turn int oa sigmoid for classification, or something else for regression, or something else for many labels?
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        
        

In [71]:
net = Net()

In [72]:
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [60]:
x = torch.randn(1, 1, 32, 32)

In [61]:
out = net(x)

In [62]:
print(out)

tensor([[-0.1042, -0.0491,  0.0138,  0.1085,  0.0212,  0.1046,  0.0260,  0.0189,
         -0.0663,  0.0476]], grad_fn=<AddmmBackward>)


In [56]:
x = torch.randn(1, 1, 32, 32)
x = x = F.relu(net.conv1(x))
x = F.max_pool2d(x, (2,2))

x = F.relu(net.conv2(x))
x = F.max_pool2d(x, 2)
x = x.view(-1, net.num_flat_features(x))
x = F.relu(net.fc1(x))
x = F.relu(net.fc2(x))
x = net.fc3(x)

print(x[0], x.shape)

# x = net.conv2(x)
# print(x[0], x.shape)

tensor([-0.0183, -0.0361, -0.0030, -0.0803,  0.0554, -0.0263, -0.0455,  0.1384,
        -0.0291, -0.1259], grad_fn=<SelectBackward>) torch.Size([1, 10])


In [49]:
# Max pooling over a (2, 2) window
x = torch.randn(1, 1, 32, 32)
x = F.max_pool2d(F.relu(net.conv1(x)), (2, 2))
# If the size is a square you can only specify a single number
x = F.max_pool2d(F.relu(net.conv2(x)), 2)
x = x.view(-1, net.num_flat_features(x))
x = F.relu(net.fc1(x))
x = F.relu(net.fc2(x))
x = net.fc3(x)
print(x[0], x.shape)

tensor([ 0.0023,  0.0020,  0.0834, -0.0199, -0.1150, -0.0363,  0.0750, -0.0309,
         0.0785,  0.0283], grad_fn=<SelectBackward>) torch.Size([1, 10])


In [70]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [69]:
net.conv1.weight

Parameter containing:
tensor([[[[ 0.0777, -0.1345,  0.2179],
          [-0.0741, -0.2793, -0.1412],
          [ 0.3285, -0.1576,  0.1425]]],


        [[[-0.0036,  0.0382, -0.1393],
          [ 0.2593,  0.1974,  0.1967],
          [-0.1425,  0.2781, -0.2777]]],


        [[[ 0.0796,  0.2798, -0.2864],
          [ 0.2809,  0.1457,  0.0103],
          [-0.1299,  0.0395, -0.1966]]],


        [[[ 0.1813, -0.2717,  0.2518],
          [-0.1234, -0.3090, -0.1879],
          [ 0.1291,  0.1899,  0.3126]]],


        [[[ 0.2703,  0.1953,  0.2264],
          [ 0.0842, -0.3084,  0.0964],
          [-0.0077,  0.2323,  0.1986]]],


        [[[-0.1336, -0.1257, -0.0574],
          [ 0.2781,  0.1996,  0.0563],
          [ 0.1192,  0.3174, -0.0979]]]], requires_grad=True)

In [68]:
list(net.parameters())[0]

Parameter containing:
tensor([[[[ 0.0777, -0.1345,  0.2179],
          [-0.0741, -0.2793, -0.1412],
          [ 0.3285, -0.1576,  0.1425]]],


        [[[-0.0036,  0.0382, -0.1393],
          [ 0.2593,  0.1974,  0.1967],
          [-0.1425,  0.2781, -0.2777]]],


        [[[ 0.0796,  0.2798, -0.2864],
          [ 0.2809,  0.1457,  0.0103],
          [-0.1299,  0.0395, -0.1966]]],


        [[[ 0.1813, -0.2717,  0.2518],
          [-0.1234, -0.3090, -0.1879],
          [ 0.1291,  0.1899,  0.3126]]],


        [[[ 0.2703,  0.1953,  0.2264],
          [ 0.0842, -0.3084,  0.0964],
          [-0.0077,  0.2323,  0.1986]]],


        [[[-0.1336, -0.1257, -0.0574],
          [ 0.2781,  0.1996,  0.0563],
          [ 0.1192,  0.3174, -0.0979]]]], requires_grad=True)

Let's try a random 32x32 input.
Note: expected input size of this net (LeNet) is 32x32. To use this net on
the MNIST dataset, please resize the images from the dataset to 32x32.



In [73]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0157, -0.0312, -0.0706,  0.0180,  0.0730,  0.0787,  0.1607,  0.0893,
          0.0633,  0.0396]], grad_fn=<AddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random
gradients:



In [74]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<div class="alert alert-info"><h4>Note</h4><p>``torch.nn`` only supports mini-batches. The entire ``torch.nn``
    package only supports inputs that are a mini-batch of samples, and not
    a single sample.

    For example, ``nn.Conv2d`` will take in a 4D Tensor of
    ``nSamples x nChannels x Height x Width``.

    If you have a single sample, just use ``input.unsqueeze(0)`` to add
    a fake batch dimension.</p></div>

Before proceeding further, let's recap all the classes you’ve seen so far.

**Recap:**
  -  ``torch.Tensor`` - A *multi-dimensional array* with support for autograd
     operations like ``backward()``. Also *holds the gradient* w.r.t. the
     tensor.
  -  ``nn.Module`` - Neural network module. *Convenient way of
     encapsulating parameters*, with helpers for moving them to GPU,
     exporting, loading, etc.
  -  ``nn.Parameter`` - A kind of Tensor, that is *automatically
     registered as a parameter when assigned as an attribute to a*
     ``Module``.
  -  ``autograd.Function`` - Implements *forward and backward definitions
     of an autograd operation*. Every ``Tensor`` operation creates at
     least a single ``Function`` node that connects to functions that
     created a ``Tensor`` and *encodes its history*.

**At this point, we covered:**
  -  Defining a neural network
  -  Processing inputs and calling backward

**Still Left:**
  -  Computing the loss
  -  Updating the weights of the network

Loss Function
-------------
A loss function takes the (output, target) pair of inputs, and computes a
value that estimates how far away the output is from the target.

There are several different
`loss functions <https://pytorch.org/docs/nn.html#loss-functions>`_ under the
nn package .
A simple loss is: ``nn.MSELoss`` which computes the mean-squared error
between the input and the target.

For example:



In [89]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6999, grad_fn=<MseLossBackward>)


In [75]:
output = net(input)

In [76]:
target = torch.randn(10)

In [82]:
output.shape

torch.Size([1, 10])

In [83]:
target.view(output.shape)

tensor([[ 0.5546, -1.0413,  1.0663,  1.3701,  1.7462, -1.5311,  0.1164, -0.3960,
         -0.4327,  0.1205]])

In [86]:
loss

tensor(1.5379, grad_fn=<MseLossBackward>)

In [100]:
loss.grad_fn.next_functions[0][0].next_functions[1][0].next_functions[0][0].next_functions

((<AccumulateGrad at 0x7fd38c67dc50>, 0),
 (<ReluBackward0 at 0x7fd38c67d3d0>, 0),
 (<TBackward at 0x7fd38c67d490>, 0))

In [101]:
def formTree(lst): 
    tree = {} 
    for item[0] in lst: 
        currTree = tree 
  
        for key in item[0][::-1]: 
            if key not in currTree: 
                currTree[key] = {} 
            currTree = currTree[key] 
              
    return tree 

Now, if you follow ``loss`` in the backward direction, using its
``.grad_fn`` attribute, you will see a graph of computations that looks
like this:

::

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

So, when we call ``loss.backward()``, the whole graph is differentiated
w.r.t. the loss, and all Tensors in the graph that has ``requires_grad=True``
will have their ``.grad`` Tensor accumulated with the gradient.

For illustration, let us follow a few steps backward:



In [106]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

Backprop
--------
To backpropagate the error all we have to do is to ``loss.backward()``.
You need to clear the existing gradients though, else gradients will be
accumulated to existing gradients.


Now we shall call ``loss.backward()``, and have a look at conv1's bias
gradients before and after the backward.



In [107]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([0.0042, 0.0017, 0.0017, 0.0086, 0.0037, 0.0181])


Now, we have seen how to use loss functions.

**Read Later:**

  The neural network package contains various modules and loss functions
  that form the building blocks of deep neural networks. A full list with
  documentation is `here <https://pytorch.org/docs/nn>`_.

**The only thing left to learn is:**

  - Updating the weights of the network

Update the weights
------------------
The simplest update rule used in practice is the Stochastic Gradient
Descent (SGD):

     ``weight = weight - learning_rate * gradient``

We can implement this using simple Python code:

.. code:: python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different
update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc.
To enable this, we built a small package: ``torch.optim`` that
implements all these methods. Using it is very simple:



In [108]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

.. Note::

      Observe how gradient buffers had to be manually set to zero using
      ``optimizer.zero_grad()``. This is because gradients are accumulated
      as explained in the `Backprop`_ section.



In [109]:
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [111]:
optimizer.zero_grad()

In [112]:
output = net(input)

In [113]:
loss = criterion(output, target)

In [114]:
loss.backward()

In [115]:
optimizer.step()

In [124]:
list(net.parameters())[0].grad.data

tensor([[[[ 0.0045, -0.0216,  0.0069],
          [-0.0107,  0.0131,  0.0012],
          [ 0.0085, -0.0075,  0.0168]]],


        [[[-0.0244,  0.0009,  0.0093],
          [-0.0047,  0.0057,  0.0020],
          [ 0.0007, -0.0058, -0.0223]]],


        [[[ 0.0025,  0.0107, -0.0040],
          [-0.0129,  0.0036, -0.0064],
          [-0.0106,  0.0052,  0.0075]]],


        [[[-0.0053, -0.0122,  0.0073],
          [ 0.0124, -0.0010, -0.0142],
          [ 0.0171,  0.0053, -0.0016]]],


        [[[-0.0005, -0.0100,  0.0043],
          [ 0.0035,  0.0067, -0.0035],
          [-0.0010, -0.0128, -0.0115]]],


        [[[ 0.0124, -0.0132,  0.0026],
          [ 0.0240, -0.0012, -0.0280],
          [-0.0062, -0.0054,  0.0088]]]])

In [119]:
for f in net.parameters():
    print(f.data.sub(f.grad.data * 0.01))

tensor([[[[ 0.1708, -0.1631,  0.1065],
          [ 0.0154, -0.2791, -0.0735],
          [-0.3141, -0.0578, -0.0067]]],


        [[[-0.0773,  0.0713, -0.2849],
          [-0.0330,  0.0992,  0.3285],
          [-0.2101, -0.2138, -0.0871]]],


        [[[-0.0972, -0.2907,  0.0486],
          [-0.1545,  0.2436, -0.1192],
          [-0.0904,  0.2773, -0.1412]]],


        [[[-0.2057, -0.2692,  0.1554],
          [-0.1270,  0.0954, -0.0358],
          [ 0.1113,  0.1724, -0.1953]]],


        [[[-0.2750, -0.2322, -0.1473],
          [-0.2119,  0.3159,  0.1333],
          [-0.1316,  0.2882,  0.0008]]],


        [[[-0.1392, -0.0272,  0.1848],
          [ 0.1091, -0.0081, -0.1786],
          [-0.0239, -0.2933,  0.0404]]]])
tensor([ 0.1022,  0.2704,  0.1789,  0.0725, -0.2507,  0.1652])
tensor([[[[ 6.3000e-02,  5.3595e-02,  1.0445e-01],
          [ 7.1985e-02, -1.0692e-01,  7.3201e-02],
          [-1.0170e-01,  7.3178e-02,  1.1409e-01]],

         [[-1.2288e-01, -1.1253e-01, -1.1343e-01],
      

SEnd an input through the model
Calculate the ypred
compute the loss
get the gradient of loss with respect to all weights
update the weights

- define the learning task
    - the appropriate sample/subject and time id
    - y[t+1,i] = F(X[t])
        - i = patient
        - t = a week
    - response variable / label
    - what is X
    - eval metrics
        - loss function
    - what is the splitting strategy?
- define a network
    - layers
    - forward pass
- send input through the network
- calculate loss vs target
- optimizer.step()